In [1]:
from PIL import Image as Im
import numpy as np
import sys
import os

In [43]:
def detect(path):
        #detects spot's border from the left
    def leftToRight(data, mask):
        for r in range (0, data.shape[0]):
            for i in range (1, data.shape[1]):
                if data[r, i] < data[r, i-1]*0.9:            
                    mask[r, i] = data[r, i]
                    break
                else:
                    i +=1
            r +=1
        return mask
    #detects spot's border from the right
    def rightToLeft(data, mask):
        for r in range (0, data.shape[0]):
            i= data.shape[1]-2
            while i > 0:
                if data[r, i] < data[r, i+1]*0.9:            
                    mask[r, i] = data[r, i]
                    break
                else:
                    i -=1
            r +=1
        return mask
    #returns extrema of spot
    def findExtr(mask):
        xmin = mask.shape[1]-1
        xmax = 0
        for r in range(0, mask.shape[0]):
            for i in range(0, mask.shape[1]):
                if (mask[r, i] !=0) & (i < xmin):
                    xmin = i
                    break
                else:
                    i +=1
            r +=1
        for r in range(0, mask.shape[0]):
            i = mask.shape[1]-1
            while i > 0:
                if (mask[r, i] > 0) & (i>xmax):
                    xmax=i
                    break
                else:
                    i -=1
            r +=1
        return xmin, xmax
    #creates dictionary with coords
    def dictCreator(filename, x, y):
        arr = [x[0], x[1], y[0], y[1]]
        dict={'file':filename, 'coords':arr}    
        return dict
    #draws rectangular
    def paint(image, dict):
        dataRGB = np.asarray(image).copy()
        for a in range(dict['coords'][0], dict['coords'][1]):
            dataRGB[dict['coords'][2], a] = [255, 0 , 0]
        for a in range(dict['coords'][0], dict['coords'][1]):
            dataRGB[dict['coords'][3], a] = [255, 0 , 0]
        for a in range(dict['coords'][2], dict['coords'][3]):
            dataRGB[a, dict['coords'][0]] = [255, 0 , 0]
            dataRGB[a, dict['coords'][1]] = [255, 0 , 0]
            a +=1
        RGB = Im.fromarray(dataRGB)
        return RGB
    
    
    #dealing with proper input
    files = []
    with os.scandir(path) as entries:
        for entry in entries:
            if entry.is_file():
                files.append(entry.name)
    output1 = []
    output2 = []
    for file in files:
        #opening image
        image = Im.open(path+file)
        #converting to greyscale
        imL = image.convert(mode='L')
        #getting numerical data from greyscale image
        data = np.asarray(imL)
        #computing spot's coordinates
        mask = np.zeros(data.shape)
        dataT = data.T
        maskT = mask.T
        mask  = leftToRight(data, mask)
        mask = rightToLeft(data, mask)
        mask = maskT.T
        maskT  = leftToRight(dataT, maskT)
        maskT = rightToLeft(dataT, maskT)
        mask = maskT.T
        dict = dictCreator(file, findExtr(mask), findExtr(maskT))
        imageFin = paint(image, dict)
        output1.append(dict)
        imageFin.save('result'+file)
        #as I do not understand how the second output should look like, I just save PNG files with borders 
        output2.append('Resulting image for '+file+' is saved as '+'result'+file+' in the root directory')
    return output1, output2
        


In [44]:
detect('./Downloads/Blobs/')

([{'file': 'SR39.png', 'coords': [6, 95, 12, 54]},
  {'file': 'SR22.png', 'coords': [7, 109, 17, 64]},
  {'file': 'SR26.png', 'coords': [9, 130, 4, 32]},
  {'file': 'SR61.png', 'coords': [7, 112, 9, 47]},
  {'file': 'SR66.png', 'coords': [7, 107, 13, 64]},
  {'file': 'SR8.png', 'coords': [8, 119, 4, 31]},
  {'file': 'SR57.png', 'coords': [8, 110, 8, 47]},
  {'file': 'SR14.png', 'coords': [10, 130, 6, 34]},
  {'file': 'SR64.png', 'coords': [7, 108, 8, 43]},
  {'file': 'SR23.png', 'coords': [10, 130, 4, 31]},
  {'file': 'SR11.png', 'coords': [10, 130, 5, 30]},
  {'file': 'SR46.png', 'coords': [7, 96, 8, 38]},
  {'file': 'SR31.png', 'coords': [7, 102, 12, 66]},
  {'file': 'SR1.png', 'coords': [6, 112, 22, 74]},
  {'file': 'SR19.png', 'coords': [7, 112, 5, 34]},
  {'file': 'SR52.png', 'coords': [6, 102, 8, 48]},
  {'file': 'SR12.png', 'coords': [7, 113, 15, 65]},
  {'file': 'SR28.png', 'coords': [10, 136, 5, 36]}],
 ['Resulting image for SR39.png is saved as resultSR39.png in the root dire